<a href="https://colab.research.google.com/github/Saitou0813/DeepLearning2022Late/blob/main/Selenium%E3%81%A7%E7%94%BB%E5%83%8F%E5%8F%8E%E9%9B%862022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##seleniumを使って自動で画像検索し、結果を収集する

##必要なアプリのインストール
Seleniumのインストール(バージョンを指定する)

In [18]:
!pip install selenium==4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### chromium-chromedriverのインストール

In [19]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (107.0.5304.87-0ubuntu11.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


###seleniumを実行して操作する

### その他のライブラリのimport

In [20]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import time
import datetime
# エラー処理用
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

### ログ用に時刻を取得

In [21]:
tm_start = time.time()# 処理時間計測用
dt_now=datetime.datetime.now()# 現在日時
dt_date_str = dt_now.strftime('%Y/%m/%d %H:%M')# 書式指定して変換
print(dt_date_str)

2022/11/29 03:24


### 検索に関する設定値を定義する

In [22]:
QUERY = '犬 フリー'# 検索したいキーワード
LIMIT_DL_NUM=100 # ダウンロード数の上限
SAVE_DIR='img/dog'# 出力フォルダ(自動生成させる)
FILE_NAME='dog'# ファイル名(後ろに連番と拡張子をつけていく)
TIMEOUT=60 #検索のタイムアウト(秒)
ACCESS_WAIT=1# アクセスする間隔(秒)←超重要
RETRY_NUM=3# クリックやrequestのリトライ回数
DRIVER_PATH='/usr/bin/chromedriver'# chromedriver.exeへのパス

## Chromeをヘッドレスモードで起動する
ヘッドレスモード = ブラウザを画面に開かないで実行するモード

In [23]:
# optionの指定を準備
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('start-fullscreen')
options.add_argument('--disable-plugins')
options.add_argument('--disable-extensions')
# 起動する
driver=webdriver.Chrome(DRIVER_PATH, options=options)
driver.implicitly_wait(TIMEOUT)# タイムアウトを設定
# ログに処理時間を出力
tm_driver=time.time()
print('WebDriver起動完了',f'{tm_driver-tm_start}s')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.


WebDriver起動完了 1.0781919956207275s


## Googleの画像検索へのアクセス

In [24]:
#QUERY条件付きでURLを定義する
url=f'https://www.google.com/search?q=%7BQUERY%7D&tbm=isch'
# 組み立てたURLでアクセスして検索
driver.get(url)
tm_geturl=time.time()
print('Google画像検索ページ取得',f'{tm_geturl-tm_driver:.1f}s')

Google画像検索ページ取得 2.0s


#結果の要素を取得していく

In [25]:
#ページのサムネイル画像の要素群を取得
#(islmpクラスが指定されたimg=サムネイルを全て拾う)
tmb_elems = driver.find_elements_by_css_selector('#islmp img')
#サムネイルのリストから各サムネイルのalt属性を取得
tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
#alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
count = len(tmb_alts) - tmb_alts.count('')
print(count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  This is separate from the ipykernel package so we can avoid doing imports until


48


## 下にスクロールさせてさらに険悪結果を表示させてまたカウント

In [26]:
#定義したLIMIT_DL_NUM(ダウンロード数の上限)になるまでスクロールを繰り返す
while count < LIMIT_DL_NUM:
  #ページの一番下へスクロールして新しいサムネイル画像を表示させる
  driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
  time.sleep(1) #スリープを繰り返す間隔(秒)を入れる　←重要
  tmb_elems = driver.find_elements_by_css_selector('#islmp img')
  #サムネイルのリストから各サムネイルのalt属性を取得
  tmb_alts = [tmb.get_attribute('alt') for tmb in tmb_elems]
  #alt属性が空(小さいサムネイル)のものを差し引いて合計を算出
  count = len(tmb_alts) - tmb_alts.count('')
  print(count)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  


100


## サムネイル画像をクリックすると右側に開かれる領域をオブジェクトに取得

In [27]:
imgframe_elem = driver.find_element_by_id(('islsp'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


### 画像へURLアクセスして保存する準備をする

In [31]:
# 画像の保存フォルダを作成
os.makedirs(SAVE_DIR, exist_ok=True)
# テキスト用のヘッダーを作る
HTTP_HEADERS = {'User-Aget':driver.execute_script('return navigator.userAgent;')}
print(HTTP_HEADERS)
# ダウンロードしたい対象のファイル拡張子(extensions)
IMG_EXTS = ('.jpg','.jpeg','.pmg','.gif')

{'User-Aget': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/107.0.5304.87 Safari/537.36'}


## ダウンロード中に利用する関数を定義する
### 画像の拡張子を取得する関数

In [32]:
# 拡張子を取得する関数
def get_extensions(url):
  url_lower = url.lower()
  for img_ext in IMG_EXTS:# 拡張子のリストから1つずつ比較
    if img_ext in url_lower:
      extension = '.jpg' if img_ext == '.jpeg' else img_ext
      break
      # if終わり
    else:
      extension = ''
    # for終わり
  return extension